# Week 3: Peer Graded Assignment
Creating a dataframe from html: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M,


In [1]:
!pip install beautifulsoup4
!pip install lxml

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: lxml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done

# All requested packages already installed.



In [4]:
import csv

html_content = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(html_content, 'lxml')
#print(soup.prettify())
table = soup.find('table', class_ ='wikitable sortable')
table_rows = table.find_all('tr')

csv_file = open('Toronto_PostalCodes.csv','w')
csv_writer=csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])
for i, tr in enumerate (table_rows):
    if (i==0):
        th = tr.find_all('th')
        header =[i.text for i in th]
        #print(header)
       # csv_writer.writerow(header)
    else:
        td = tr.find_all('td')
        row =[i.text for i in td]
        #print(row)
        csv_writer.writerow(row)
        
csv_file.close()

In [5]:
# Reading the csv data
df=pd.read_csv('Toronto_PostalCodes.csv')

In [6]:
# stripping the '\n' character from the Neighbourhood column
df['Neighbourhood'] =df['Neighbourhood'].str.strip('\n')

# Replacing all Not assigned with numpy NaN
df.replace('Not assigned', np.nan, inplace=True)

# dropping the rows which has Borough as NaN
new_df = df.dropna(subset=['Borough'], axis=0)

#Setting the neighbourhood as Borough's name whose Neighbourhood are NaN
clean_df = new_df.fillna(method='ffill', axis ='columns')
clean_df.head(15)

# Joining the Neighbourhoods with ',' of same borough
gp_df = clean_df[['Postcode','Borough','Neighbourhood']]
df2 = gp_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df2.shape

(103, 3)

## Using the geocoder
 using the geocoder arcgis to get the latitude and longitude of all postal codes.

In [8]:
!conda install -c conda-forge geocoder --yes
import geocoder # import geocoder

Solving environment: done

# All requested packages already installed.



In [9]:
# using the geocoder arcgis to get the latitude and longitude of all postal codes.

Postcode_latitude=[]
Postcode_longitude =[]

for i,postcode in enumerate(df2['Postcode']):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(df2['Postcode'][i]))
    lat_lng_coords = g.latlng
    Postcode_latitude.append(g.latlng[0])
    Postcode_longitude.append(g.latlng[1])
    
    
df2['Latitude'] = Postcode_latitude
df2['Longitude'] = Postcode_longitude

df2.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163


In [10]:
df2.shape

(103, 5)

# Neighbourhood Clustering in Toronto using Folium

In [11]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium     # map rendering library

Solving environment: done

# All requested packages already installed.



## Map of Toronto with Boroughs superimposed on top

In [12]:
Toronto_lat = 43.6532
Toronto_long = -79.3832

# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[Toronto_lat, Toronto_long], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        legend_name ='Map showing the different Postal Code around Toronto',
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [13]:
downtown_toronto_data = df2[df2['Borough'] =='Downtown Toronto'].reset_index(drop=True)
downtown_toronto_data.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.668160,-79.366602
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180
5,M5C,Downtown Toronto,St. James Town,43.651210,-79.375481
6,M5E,Downtown Toronto,Berczy Park,43.645160,-79.373675
7,M5G,Downtown Toronto,Central Bay Street,43.656091,-79.384930
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.649515,-79.382503
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.623470,-79.391507


Let's visualize the Downtown Toronto's map with its neighbourhood on top.

In [14]:
# Got the latitude and longitude of Downtown Toronto directly from the web
Lat = 43.6543
Long = -79.3860

map_downtown_toronto = folium.Map(location=[Lat, Long], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude'], downtown_toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

### Define Foursquare credential and version

In [15]:
# The code was removed by Watson Studio for sharing.

In [16]:
# Getting the first neighbourhood in our dataframe
downtown_toronto_data.loc[0, 'Neighbourhood']

'Rosedale'

### Now let's get the top 100 venues within 500m radius around Rosedale

In [17]:
Rosedale_lat = downtown_toronto_data.loc[0,'Latitude']
Rosedale_long = downtown_toronto_data.loc[0,'Longitude']
LIMIT = 100
Radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, CLIENT_SECRET, VERSION, Rosedale_lat,Rosedale_long, Radius,LIMIT)

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd47877351e3d1d4fd47de0'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4d6d1a35cf7e41bd9ec08285-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/financial_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d10a951735',
         'name': 'Bank',
         'pluralName': 'Banks',
         'primary': True,
         'shortName': 'Bank'}],
       'id': '4d6d1a35cf7e41bd9ec08285',
       'location': {'address': '408 Summerhill Avenue',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 487,
        'formattedAddress': ['408 Summerhill Avenue',
         'Toronto ON M4W 2E4',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.68609399737299,
  

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [21]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,TD Canada Trust,Bank,43.686094,-79.376549
1,Rosedale Park,Playground,43.682328,-79.378934
2,Rosedale Tennis Club,Tennis Court,43.683226,-79.378984
3,Whitney Park,Park,43.682036,-79.373788
4,Mooredale House,Building,43.678631,-79.380091


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


Let's create a function to repeat the same process to all the Downtown Toronto postcodes.

In [23]:
def getNearbyVenues(postcodes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for postcode, lat, lng in zip(postcodes, latitudes, longitudes):
                   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            postcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
# Getting nearvy venues for all postcodes in Downtown Toronto

downtown_toronto_venues = getNearbyVenues(postcodes=downtown_toronto_data['Postcode'],
                                   latitudes=downtown_toronto_data['Latitude'],
                                   longitudes=downtown_toronto_data['Longitude']
                                  )

In [25]:
print(downtown_toronto_venues.shape)
downtown_toronto_venues.head(20)

(1236, 7)


,Postcode,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4W,43.68194,-79.378474,TD Canada Trust,43.686094,-79.376549,Bank
1,M4W,43.68194,-79.378474,Rosedale Park,43.682328,-79.378934,Playground
2,M4W,43.68194,-79.378474,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
3,M4W,43.68194,-79.378474,Whitney Park,43.682036,-79.373788,Park
4,M4W,43.68194,-79.378474,Mooredale House,43.678631,-79.380091,Building
5,M4X,43.66816,-79.366602,F'Amelia,43.667536,-79.368613,Italian Restaurant
6,M4X,43.66816,-79.366602,Cranberries,43.667843,-79.369407,Diner
7,M4X,43.66816,-79.366602,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
8,M4X,43.66816,-79.366602,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
9,M4X,43.66816,-79.366602,Murgatroid,43.667381,-79.369311,Restaurant


In [26]:
# Let's check how many venues we got for each Postcode
downtown_toronto_venues.groupby('Postcode').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
M4W,5,5,5,5,5,5
M4X,40,40,40,40,40,40
M4Y,83,83,83,83,83,83
M5A,20,20,20,20,20,20
M5B,100,100,100,100,100,100
M5C,100,100,100,100,100,100
M5E,64,64,64,64,64,64
M5G,100,100,100,100,100,100
M5H,100,100,100,100,100,100


In [27]:
# Printing the unique categories of venues
print('There are {} uniques categories.'.format(len(downtown_toronto_venues['Venue Category'].unique())))

There are 180 uniques categories.


###  Analyzing each neighbourhood

In [28]:
# one hot encoding
downtown_toronto_onehot = pd.get_dummies(downtown_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add Postcode column back to dataframe
downtown_toronto_onehot['Postcode'] = downtown_toronto_venues['Postcode'] 

# move Postcode column to the first column
fixed_columns = [downtown_toronto_onehot.columns[-1]] + list(downtown_toronto_onehot.columns[:-1])
downtown_toronto_onehot = downtown_toronto_onehot[fixed_columns]

downtown_toronto_onehot.head(20)

,Postcode,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4W,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,M4X,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,M4X,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,M4X,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,M4X,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,M4X,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
downtown_toronto_onehot.shape

(1236, 181)

#### Next, let's group rows by Postcode and by taking the mean of the frequency of occurrence of each category¶

In [30]:
downtown_toronto_grouped = downtown_toronto_onehot.groupby('Postcode').mean().reset_index()
downtown_toronto_grouped

,Postcode,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,M4W,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
1,M4X,0.000000,0.000000,0.025000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
2,M4Y,0.012048,0.012048,0.012048,0.000000,0.00,0.012048,0.000000,0.000000,0.000000,...,0.012048,0.00000,0.00000,0.000000,0.000000,0.012048,0.012048,0.00000,0.012048,0.000000
3,M5A,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.050000
4,M5B,0.000000,0.000000,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.01000,0.000000,0.010000,0.010000,0.010000,0.01000,0.000000,0.000000
5,M5C,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,...,0.000000,0.00000,0.00000,0.000000,0.010000,0.000000,0.000000,0.00000,0.000000,0.000000
6,M5E,0.000000,0.000000,0.000000,0.015625,0.00,0.000000,0.000000,0.000000,0.015625,...,0.000000,0.00000,0.00000,0.000000,0.015625,0.000000,0.000000,0.00000,0.000000,0.000000
7,M5G,0.000000,0.000000,0.010000,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.01000,0.000000,0.010000,0.010000,0.010000,0.01000,0.000000,0.000000
8,M5H,0.000000,0.000000,0.030000,0.010000,0.00,0.000000,0.030000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.010000,0.000000,0.000000,0.01000,0.000000,0.000000
9,M5J,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,...,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000


First, let's write a function to sort the venues in descending order and then put into dataframe

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcodes_venues_sorted = pd.DataFrame(columns=columns)
postcodes_venues_sorted['Postcode'] = downtown_toronto_grouped['Postcode']

for ind in np.arange(downtown_toronto_grouped.shape[0]):
    postcodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_toronto_grouped.iloc[ind, :], num_top_venues)

postcodes_venues_sorted.head()

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Playground,Park,Bank,Building,Tennis Court,Deli / Bodega,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm
1,M4X,Coffee Shop,Market,Bakery,Italian Restaurant,Café,Pizza Place,Restaurant,Snack Place,Butcher,Liquor Store
2,M4Y,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Gastropub,Nightclub,Pub,Bubble Tea Shop,Men's Store
3,M5A,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Spa,Event Space,Food Truck,Electronics Store,Restaurant,Mexican Restaurant
4,M5B,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Italian Restaurant,Bookstore,Restaurant,Lingerie Store


#### Clustering the Postcodes into 4 clusters

In [33]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 4

downtown_toronto_grouped_clustering = downtown_toronto_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
postcodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_toronto_merged = downtown_toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_toronto_merged = downtown_toronto_merged.join(postcodes_venues_sorted.set_index('Postcode'), on='Postcode')

downtown_toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474,2,Playground,Park,Bank,Building,Tennis Court,Deli / Bodega,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.668160,-79.366602,0,Coffee Shop,Market,Bakery,Italian Restaurant,Café,Pizza Place,Restaurant,Snack Place,Butcher,Liquor Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302,0,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Gastropub,Nightclub,Pub,Bubble Tea Shop,Men's Store
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655120,-79.362640,0,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Spa,Event Space,Food Truck,Electronics Store,Restaurant,Mexican Restaurant
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657363,-79.378180,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Italian Restaurant,Bookstore,Restaurant,Lingerie Store


In [35]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [36]:
# create map
Lat = 43.6543
Long = -79.3860
map_clusters = folium.Map(location=[Lat, Long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_toronto_merged['Latitude'], downtown_toronto_merged['Longitude'], downtown_toronto_merged['Postcode'], downtown_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [37]:
downtown_toronto_merged.loc[downtown_toronto_merged['Cluster Labels'] == 0, downtown_toronto_merged.columns[[1] + list(range(5, downtown_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,0,Coffee Shop,Market,Bakery,Italian Restaurant,Café,Pizza Place,Restaurant,Snack Place,Butcher,Liquor Store
2,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Gastropub,Nightclub,Pub,Bubble Tea Shop,Men's Store
3,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Spa,Event Space,Food Truck,Electronics Store,Restaurant,Mexican Restaurant
4,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Italian Restaurant,Bookstore,Restaurant,Lingerie Store
5,Downtown Toronto,0,Coffee Shop,Restaurant,Hotel,Café,Bakery,Italian Restaurant,Cosmetics Shop,Seafood Restaurant,Breakfast Spot,Gastropub
6,Downtown Toronto,0,Coffee Shop,Restaurant,Cocktail Bar,Hotel,Lounge,Cheese Shop,Breakfast Spot,Steakhouse,Beer Bar,Seafood Restaurant
7,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Sushi Restaurant,Middle Eastern Restaurant,Tea Room,Restaurant,Chinese Restaurant,Bakery,Fast Food Restaurant
8,Downtown Toronto,0,Café,Coffee Shop,Hotel,Bakery,Japanese Restaurant,Deli / Bodega,Restaurant,Burger Joint,Steakhouse,Bar
10,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Italian Restaurant,Deli / Bodega,Gastropub,American Restaurant,Steakhouse
11,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Bakery,Gym,Italian Restaurant,Japanese Restaurant,Deli / Bodega
